In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import warnings

warnings.filterwarnings("ignore")

import lightgbm as lgb
import optuna
import ray
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    roc_auc_score,
    balanced_accuracy_score,
)
from imblearn.over_sampling import RandomOverSampler, SMOTE, SMOTENC
from sklearn.utils import compute_class_weight, class_weight
from sklearn.manifold import Isomap
from typing import Tuple
from scipy.special import expit
import xgboost as xgb

In [2]:
train = pd.read_csv('train.csv')
train['EJ'].replace(['A', 'B'], [1, 0], inplace=True)

ej = np.array(train['EJ']).reshape(-1, 1)

sample_submission = pd.read_csv('sample_submission.csv')

y = np.array(train['Class'])

greeks = pd.read_csv('greeks.csv')
greeks['Epsilon'].replace(['Unknown'], np.nan, inplace=True)

train_greeks = pd.merge(train, greeks, on='Id')
greek_columns = greeks.columns.drop(['Id', 'Epsilon', 'Alpha']).tolist()

In [3]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# scaler = MinMaxScaler()
scaler = StandardScaler()

x_numerical_columns = train.drop(columns=["Id", "Class", "EJ"]).columns.tolist()
x_categorical_columns = ["EJ"]
x_cols = x_numerical_columns + x_categorical_columns

scaler.fit(train_greeks[x_numerical_columns])
x_standardized = scaler.transform(train_greeks[x_numerical_columns])

In [4]:
from sklearn.impute import KNNImputer

knn = KNNImputer()
knn.fit(x_standardized)
x_imputed_standardized = knn.transform(x_standardized)

X = np.concatenate((x_imputed_standardized, ej), axis=1)

In [5]:
def to_df(arr):
    cols = x_numerical_columns + x_categorical_columns
    df = pd.DataFrame(arr, columns=cols)
    df[x_categorical_columns] = df[x_categorical_columns].astype("category")
    return df

In [10]:
def balancedlogloss(
    predt: np.ndarray, dtrain: lgb.Dataset
) -> Tuple[np.ndarray, np.ndarray]:
    y = dtrain.get_label()
    n0 = len(y[y == 0])
    n1 = len(y[y == 1])

    p = expit(predt)
    p[p == 0] = 1e-15

    grad = 1 / 2 * ((1 - y) / (1 - p) - y / p)
    hess = 1 / 2 * ((1 - y) / ((1 - p) ** 2) + y / (p**2))
    return grad, hess


def balancedlogloss_eval_lgb(
    predt: np.ndarray, dtrain: lgb.Dataset
) -> Tuple[np.ndarray, np.ndarray]:
    y = dtrain.get_label()
    n0 = len(y[y == 0])
    n1 = len(y[y == 1])
    p = expit(predt)

    p[p == 0] = 1e-15

    return (
        "balanced_logloss",
        (-1 / n0 * (sum((1 - y) * np.log(1 - p))) - 1 / n1 * (sum(y * np.log(p)))) / 2,
        True
    )

def balancedlogloss_eval_xgb(
    predt: np.ndarray, dtrain: lgb.Dataset
) -> Tuple[np.ndarray, np.ndarray]:
    y = dtrain.get_label()
    n0 = len(y[y == 0])
    n1 = len(y[y == 1])
    p = expit(predt)

    p[p == 0] = 1e-15

    return (
        "balanced_logloss",
        (-1 / n0 * (sum((1 - y) * np.log(1 - p))) - 1 / n1 * (sum(y * np.log(p)))) / 2,
    )

In [18]:
def cv(X_kf, y_kf, lgb_param, num_boost_round=100):
    
    train_evals = pd.DataFrame()
    test_evals = pd.DataFrame()

    kf = StratifiedKFold(10, random_state=3, shuffle=True)
    k = 0

    for train_index, test_index in kf.split(X_kf, y_kf):
        X_train = pd.DataFrame(X_kf[train_index], columns=x_cols)
        X_test = pd.DataFrame(X_kf[test_index], columns=x_cols)
        y_train = y_kf[train_index]
        y_test = y_kf[test_index]

        # sampler = RandomOverSampler()
        sampler = SMOTE()
        X_train, y_train = sampler.fit_resample(X_train, y_train)

        n_components = 40
        dr_cols = ['Isomap'+str(i) for i in range(n_components)]
        dr = Isomap(n_components=n_components)
        dr.fit(X_train)
        X_train_dr = pd.DataFrame(dr.transform(X_train), columns=dr_cols)
        X_test_dr = pd.DataFrame(dr.transform(X_test), columns=dr_cols)

        train_set = lgb.Dataset(X_train_dr, y_train)
        test_set = lgb.Dataset(X_test_dr, y_test)

        # X_train = pd.concat([X_train, X_train_dr], axis=1)
        # X_test = pd.concat([X_test, X_test_dr], axis=1)
        # cols = X_train.columns.tolist()

        # train_set = lgb.Dataset(X_train, y_train, feature_name=cols)
        # test_set = lgb.Dataset(X_test, y_test, feature_name=cols)

        evals = {}
        lgb_model = lgb.train(
                        params=lgb_param,
                        train_set=train_set,
                        # categorical_feature=['EJ'],
                        valid_sets=[train_set, test_set],
                        verbose_eval=False,
                        fobj=balancedlogloss,
                        feval=balancedlogloss_eval_lgb,
                        num_boost_round=num_boost_round,
                        evals_result=evals
                        )

        train_evals[str(k)] = evals["training"]["balanced_logloss"]
        test_evals[str(k)] = evals["valid_1"]["balanced_logloss"]
        k = k + 1

    eval_df = pd.concat([train_evals.mean(axis=1, skipna=False), test_evals.mean(axis=1, skipna=False)], axis=1)
    eval_df.columns = ['train', 'test']
    return eval_df

lgb_param = {'learning_rate': 0.1,
             'subsample': 0.7,
             'max_depth': 4,
             'lambda_l1': 30,
             'lambda_l2': 30,
             'verbosity': -1
            }


cv_results = cv(X, y, lgb_param, num_boost_round=400)

print(cv_results['test'].min())
cv_results.iloc[range(0, len(cv_results), 50)]


0.5414611634858639


,train,test
0,0.682868,0.687094
50,0.359749,0.541789
100,0.252408,0.594782
150,0.211499,0.712240
200,0.192777,0.865528
250,0.181095,1.025092
300,0.174093,1.184912
350,0.170040,1.368567


In [ ]:
from itertools import product
from sklearn.model_selection import ParameterGrid

param_space = {
    'num_leaves': [10, 50, 100, 300],
    'max_depth': [5, 30, 50, 70],
    'lambda_l1': [0.02, 0.5, 10],
    'lambda_l2': [30], 
    'verbosity': [-1]
}

grid = ParameterGrid(param_space)

for params_grid in grid:
    print(params_grid)

    cv_results = cv(X, y, params_grid, num_boost_round=400)
    print(cv_results['test'].min())

In [ ]:
""" test = pd.read_csv("test.csv")
test["EJ"].replace(["A", "B"], [1, 0], inplace=True)
test_ej = np.array(test["EJ"]).reshape(-1, 1)

x_test_scaled = scaler.transform(test[x_numerical_columns])

X_test = np.append(x_test_scaled, test_ej, axis=1) """